In [1]:
from elasticsearch import Elasticsearch
client = Elasticsearch()

In [ ]:
query = {
    'bool': 
    {
        'must':
        {
            'match': 
            {
                'response_code':403,
            }
        },
        'filter':
        {
            'range': 
            {
                'start_time': 
                {
                    'gte': '2022-05-15'
                }
            }
        }
    }
}

In [9]:
query = {
    'bool': 
    {
        'must':
        [
            {
                    "bool": {
                        "should": [
                            {"term": {'response_code':403}},
                            {"term": {'response_code':500}}
                        ]
                    }
                }
        ],
        'filter':
        {
            'range': 
            {
                'start_time': 
                {
                    'gte': '2022-05-15'
                }
            }
        }
    }
}
response = client.search(
    index="envoy-k8s",
    query=query,
    sort={"start_time": {"order": "desc"}},
    size = 1000
)

In [10]:
response['hits']

{'total': {'value': 106, 'relation': 'eq'},
 'max_score': None,
 'hits': [{'_index': 'envoy-k8s',
   '_type': '_doc',
   '_id': 'AlPI7oABUy71rRX8OHix',
   '_score': None,
   '_source': {'start_time': '2022-05-23T02:37:57.858Z',
    'method': 'GET',
    'x_envoy_original_path': '/productpage',
    'protocol': 'HTTP/1.1',
    'response_code': '500',
    'response_flags': 'RLSE',
    'response_code_details': 'rate_limiter_error',
    'connection_termination_details': '-',
    'upstream_failure_reason': '-',
    'bytes_received': '0',
    'bytes_sent': '0',
    'duration': '0',
    'x_envoy_upstream_service_time': '-',
    'x_forwarded_for': '10.244.1.1',
    'user_agent': 'curl/7.68.0',
    'x_request_id': '68ba8576-9854-9f2e-9f01-2d8584bb367e',
    'authority': '172.18.0.2:31691',
    'upstream_host': '-',
    'upstream_cluster': 'outbound|9080||productpage.default.svc.cluster.local',
    'upstream_local_address': '-',
    'downstream_local_address': '10.244.1.28:8080',
    'downstream_r

In [11]:
ips = {}
for hit in response['hits']['hits']:
  source = hit['_source']
  x_forwarded_for = source['x_forwarded_for']
  print(source['response_code'])
  if source['response_code'] == '403':
    count = ips.get(x_forwarded_for, 0)
    count = count + 1
    ips[x_forwarded_for] = count

print(f"total: {response['hits']['total']['value']}")
for ips, count in ips.items():
  print(f"{ips}: {count}")

500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
total: 106
127.0.0.127,192.168.1.1,192.168.1.2,10.244.1.1: 15
127.0.0.127,10.244.1.1,10.244.1.1: 2
127.0.0.127,10.244.1.1: 3
10.244.1.1: 45
192.168.1.11,10.244.1.1: 2
10.244.1.1,10.244.1.1: 4
56.5.6.7, 72.9.5.6, 98.1.2.3,10.244.1.1: 15


In [5]:
client.info()

{'name': 'elasticsearch',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'olYMaE7kRSmPJJ5kOuHg4g',
 'version': {'number': '7.9.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '083627f112ba94dffc1232e8b42b73492789ef91',
  'build_date': '2020-09-01T21:22:21.964974Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}